In [7]:
import pandas as pd

raw = pd.read_csv('Quan_FinTx.csv')

raw.head()

,name,number,machinename,WeekNum,calendardate,trandate,ChildTran,ParentTran,dept,register,...,0.05c,0.10c,0.25c,1.00c,5.00c,10.00n,20.00n,50.00n,100.00n,total
0,Walmart,58,QF-00754,23,5/29/2022,5/29/2022 6:19,CashDispensed,TillCheckout,Fuel,#88 - FUEL,...,0,0,0,-30,-10,0,-6,0,0,200.0
1,Walmart,58,QF-00754,23,5/29/2022,5/29/2022 6:20,CashDispensed,TillCheckout,Fuel,#89 - FUEL,...,0,0,0,-30,-18,0,-4,0,0,200.0
2,Walmart,58,QF-00754,23,5/29/2022,5/29/2022 11:53,CashAccepted,CashPickup,Fuel,#88 - FUEL,...,0,0,0,30,0,0,20,2,5,1030.0
3,Walmart,58,QF-00754,23,5/29/2022,5/29/2022 13:26,CashAccepted,CashPickup,Fuel,#88 - FUEL,...,0,0,0,56,16,0,17,3,2,826.0
4,Walmart,58,QF-00754,23,5/29/2022,5/29/2022 14:15,CashAccepted,CashPickup,Fuel,#88 - FUEL,...,0,0,0,0,0,0,18,0,1,460.0


In [54]:
from audioop import avg
from statistics import mean
import pandas as pd

raw = pd.read_csv('Quan_FinTx.csv')

raw.calendardate = pd.to_datetime(raw['calendardate'])

UsageByDay = raw.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum',
                          'calendardate']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()

MaxByWeek = UsageByDay.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum']).agg({

                              '0.01c':   min,
                              '0.05c':   min,
                              '0.10c':   min,
                              '0.25c':   min,
                              '1.00n':   min,
                              '5.00n':   min,
                              '10.00n':  min,
                              '20.00n':  min,
                              '50.00n':  min,
                              '100.00n': min
                          }).reset_index()


            
AvgByWeek = UsageByDay.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum']).agg({
                              '0.01c':   mean,
                              '0.05c':   mean,
                              '0.10c':   mean,
                              '0.25c':   mean,
                              '1.00n':   mean,
                              '5.00n':   mean,
                              '10.00n':  mean,
                              '20.00n':  mean,
                              '50.00n':  mean,
                              '100.00n': mean
                          }).reset_index()

Dispense = raw[raw['ChildTran'] == 'CashDispensed']

DispenseWeeklyMax = Dispense.groupby(['name',
                                      'number',
                                      'machinename',
                                      'WeekNum']).agg({
                                          '0.01c':   min,
                                          '0.05c':   min,
                                          '0.10c':   min,
                                          '0.25c':   min,
                                          '1.00n':   min,
                                          '5.00n':   min,
                                          '10.00n':  min,
                                          '20.00n':  min,
                                          '50.00n':  min,
                                          '100.00n': min
                                      }).reset_index()

DispenseWeeklyAvg = Dispense.groupby(['name',
                 'number',
                  'machinename',
                  'WeekNum']).agg({
                      '0.01c':   mean,
                      '0.05c':   mean,
                      '0.10c':   mean,
                      '0.25c':   mean,
                      '1.00n':   mean,
                      '5.00n':   mean,
                      '10.00n':  mean,
                      '20.00n':  mean,
                      '50.00n':  mean,
                      '100.00n': mean
                  }).reset_index()



#Format and Merge Dataframes

UsageMerge = MaxByWeek.merge(AvgByWeek, how='inner',on=['name','number','machinename','WeekNum'], suffixes=('_max','_avg'))

DispenseMerge  = DispenseWeeklyMax.merge(DispenseWeeklyAvg, how='inner',on=['name','number','machinename','WeekNum'], suffixes=('_max','_avg'))




with pd.ExcelWriter('TetheredQuantumUsage.xlsx') as writer:
    raw.to_excel(writer, sheet_name='RAW', index=False)
    UsageByDay.to_excel(writer, sheet_name='Usage By Day', index=False)
    UsageMerge.to_excel(writer, sheet_name='Weekly Usage Metrics', index=False, float_format='%.0f')
    DispenseMerge.to_excel(writer, sheet_name='Weekly Dispense Metrics', index=False, float_format='%.0f')
    
                    


In [45]:
Dispense.groupby(['name',
                 'number',
                  'machinename',
                  'WeekNum']).agg({
                      '0.01c':   mean,
                      '0.05c':   mean,
                      '0.10c':   mean,
                      '0.25c':   mean,
                      '1.00c':   mean,
                      '5.00c':   mean,
                      '10.00n':  mean,
                      '20.00n':  mean,
                      '50.00n':  mean,
                      '100.00n': mean
                  }).reset_index()

,name,number,machinename,WeekNum,0.01c,0.05c,0.10c,0.25c,1.00c,5.00c,10.00n,20.00n,50.00n,100.00n
0,Sam's Club Liquor Box,4754,QR-00005,23,-30.508475,-12.203390,-15.254237,-12.203390,-8.050847,-3.677966,0.0,-1.949153,0.0,0.0
1,Sam's Club Liquor Box,4754,QR-00005,24,-26.470588,-10.588235,-13.235294,-10.882353,-7.279412,-3.779412,0.0,-2.088235,0.0,0.0
2,Sam's Club Liquor Box,4754,QR-00005,25,-26.865672,-10.746269,-13.432836,-11.283582,-7.164179,-3.910448,0.0,-1.985075,0.0,0.0
3,Sam's Club Liquor Box,4754,QR-00005,26,-23.986111,-9.375000,-12.847222,-10.777778,-7.402778,-3.763889,0.0,-2.402778,0.0,0.0
4,Sam's Club Liquor Box,4754,QR-00005,27,-23.452055,-9.876712,-12.438356,-9.849315,-6.095890,-4.000000,0.0,-2.068493,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,Walmart,7357,QF-00992,27,-11.458333,0.000000,0.000000,-2.291667,-8.333333,-3.583333,0.0,-2.708333,0.0,0.0
3324,Walmart,7357,QF-00992,28,-16.129032,-2.580645,-1.612903,-4.354839,-6.903226,-3.387097,0.0,-2.096774,0.0,0.0
3325,Walmart,7357,QF-00992,29,-20.517241,-4.137931,-3.103448,-3.620690,-5.862069,-3.793103,0.0,-2.034483,0.0,0.0
3326,Walmart,7357,QF-00992,30,-9.718750,0.000000,0.000000,-6.250000,-8.218750,-4.250000,0.0,-1.937500,0.0,0.0


In [49]:
UsageMerge.columns.str.replace([''])

Index(['name', 'number', 'machinename', 'WeekNum', '0.01c_max', '0.05c_max',
       '0.10c_max', '0.25c_max', '1.00c_max', '5.00c_max', '10.00n_max',
       '20.00n_max', '50.00n_max', '100.00n_max', '0.01c_avg', '0.05c_avg',
       '0.10c_avg', '0.25c_avg', '1.00c_avg', '5.00c_avg', '10.00n_avg',
       '20.00n_avg', '50.00n_avg', '100.00n_avg'],
      dtype='object')

In [46]:

DispenseWeeklyMax.merge(DispenseWeeklyAvg, how='inner',on=['name','number','machinename','WeekNum'], suffixes=('_max','_avg'))

NameError: name 'DispenseWeeklyMax' is not defined